This script calculates the Climate Shelter Index for each public green area.  
A climate shelter should have these characteristics:
- Surface area > 0.5 ha 
- NDVI > 0.4 
- Good accessibility
- Presence of drinking fountains 
- Presence of  benches 
- Presence of picnic tables

### 0. Libraries

In [ ]:
import geopandas as gpd
import pandas as pd

### 1. Climate Shelter Index - CSI

In [9]:
# Load your shapefile
green_area = gpd.read_file(r'D:\Climate_Shelter_Index\isocrona\access_index')

In [10]:
green_area.head()

,quart,nome,ubicazione,classe_uni,classe_gia,classe_pen,classe_suo,area_prato,area_ug,data_agg,...,benches,id_bench,NDVI,n_cycleway,n_footway,busstop5,busstop10,disab_p_5,access_ind,geometry
0,Santo Stefano,PARCO DEI CALANCHI DI SABBIUNO,Via di Sabbiuno,PARCO ESTENSIVO,SOMMITALE,0 -20%,ARGILLOSO COMPATTO,9509.007,12461.168,2009-01-29,...,no,N/A,0.542155,0,0,2,3,-1,0.012821,"POLYGON ((11.31509 44.42248, 11.31511 44.42256..."
1,Santo Stefano,GIARDINO MONUMENTO CADUTI DI SABBIUNO,Via di Sabbiuno,PARCO,SOMMITALE,0 -20%,ARGILLOSO COMPATTO,5514.205,6730.89,2009-01-29,...,no,N/A,0.459359,0,0,0,0,-1,0.000000,"POLYGON ((11.31288 44.42104, 11.31289 44.42109..."
2,Santo Stefano,PARCO DEI CALANCHI DI SABBIUNO,Via di Sabbiuno,PARCO ESTENSIVO,SOMMITALE,0 -20%,ARGILLOSO COMPATTO,16811.194,24817.745,2007-03-13,...,no,N/A,0.624308,0,0,2,3,-1,0.012821,"POLYGON ((11.31463 44.42146, 11.31496 44.42149..."
3,Santo Stefano,PARCO PADERNO,Via Paderno,PARCO ESTENSIVO,PIANO,20 - 40%,MEDIO IMPASTO,87.272,6250.574,2007-09-20,...,no,N/A,0.877684,0,8,3,8,-1,0.071077,"POLYGON ((11.32362 44.45097, 11.32372 44.45110..."
4,Santo Stefano,PARCO PADERNO,Via Paderno,PARCO ESTENSIVO,PIANO,20 - 40%,MEDIO IMPASTO,-4.817,21360.397,2007-09-20,...,no,N/A,0.879811,0,8,3,8,-1,0.071077,"POLYGON ((11.32160 44.45340, 11.32164 44.45341..."


In [11]:
green_area.columns

Index(['quart', 'nome', 'ubicazione', 'classe_uni', 'classe_gia', 'classe_pen',
       'classe_suo', 'area_prato', 'area_ug', 'data_agg', 'siepi', 'bosco',
       'arboreo', 'arbustivo', 'pavim', 'idro', 'quartiere', 'area_stati',
       'zona_pross', 'area_Ha', 'd_fountain', 'id_df', 'picnic_tab', 'id_pt',
       'benches', 'id_bench', 'NDVI', 'n_cycleway', 'n_footway', 'busstop5',
       'busstop10', 'disab_p_5', 'access_ind', 'geometry'],
      dtype='object')

In [12]:
# Define weights for each characteristic (surface area, NDVI, accessibility, drinking fountains, benches, picnic tables)
weights = {
    'surface_area': 0.25,
    'NDVI': 0.25,
    'accessibility': 0.2,
    'drinking_fountains': 0.1,
    'benches': 0.1,
    'picnic_tables': 0.1
}


In [13]:
# Count variables for presence of drinking fountains, benches, and picnic tables
green_area['n_df'] = green_area['id_df'].apply(lambda x: len(x.split(',')) if x != 'N/A' else 0)
green_area['n_bench'] = green_area['id_bench'].apply(lambda x: len(x.split(',')) if x != 'N/A' else 0)
green_area['n_pt'] = green_area['id_pt'].apply(lambda x: len(x.split(',')) if x != 'N/A' else 0)


In [14]:
# Define the columns you want to normalize
columns_to_normalize = ['n_df', 'n_bench', 'n_pt']

# Perform min-max normalization for each column
for col_name in columns_to_normalize:
    min_value = green_area[col_name].min()
    max_value = green_area[col_name].max()
    green_area[col_name] = (green_area[col_name] - min_value) / (max_value - min_value)


In [15]:
# Normalize surface area (0.5 ha threshold)
green_area['norm_area'] = green_area['area_Ha'].apply(lambda x: 1 if x >= 0.5 else 0)


In [16]:
# Normalize NDVI (0.4 threshold)
green_area['norm_NDVI'] = green_area['NDVI'].apply(lambda x: x if x >= 0.4 else 0)


In [17]:
# Calculate the CSI based on the weighted and normalized variables
green_area['CSI'] = (
    weights['surface_area'] * green_area['norm_area'] +
    weights['NDVI'] * green_area['norm_NDVI'] +
    weights['accessibility'] * green_area['access_ind'] +
    weights['drinking_fountains'] * green_area['n_df'] +
    weights['benches'] * green_area['n_bench'] +
    weights['picnic_tables'] * green_area['n_pt']
)

In [22]:
green_area = green_area.drop(columns=['n_df', 'n_bench', 'n_pt', 'norm_area', 'norm_NDVI'])

In [23]:
green_area[green_area['NDVI']<0.41].head()

,quart,nome,ubicazione,classe_uni,classe_gia,classe_pen,classe_suo,area_prato,area_ug,data_agg,...,id_bench,NDVI,n_cycleway,n_footway,busstop5,busstop10,disab_p_5,access_ind,geometry,CSI
220,Borgo Panigale - Reno,GIARDINO TERESA NOCE,Via del Beccaccino,GIARDINO,PIANO,NaN,ARGILLOSO COMPATTO,0.114,239.302,2015-06-10,...,N/A,0.398521,0,5,9,34,22,0.125866,"POLYGON ((11.29106 44.49354, 11.29112 44.49352...",0.025173
311,Porto - Saragozza,SCUOLA MEDIA GANDINO,Via Graziano,VERDE SCOLASTICO,PIANO,0 -20%,ARGILLOSO COMPATTO,55.935,55.935,2006-06-06,...,N/A,0.388157,0,0,0,0,-1,0.000000,"POLYGON ((11.33103 44.50035, 11.33104 44.50038...",0.000000
317,Porto - Saragozza,SCUOLA ELEM. TERESINA GUIDI,Via Calori,VERDE SCOLASTICO,PIANO,0 -20%,ARGILLOSO COMPATTO,30.760,30.76,2006-06-06,...,N/A,0.378452,0,0,7,38,40,0.102983,"POLYGON ((11.33043 44.50045, 11.33043 44.50046...",0.020597
330,Santo Stefano,GIARDINO LEGNANI PIZZARDI,Via del Cane 3/b,GIARDINO,PIANO,0 -20%,MEDIO IMPASTO,3.783,7.242,2009-10-20,...,N/A,0.351903,0,0,0,0,-1,0.000000,"POLYGON ((11.34252 44.49087, 11.34252 44.49086...",0.000000
332,Santo Stefano,GIARDINO LEGNANI PIZZARDI,Via del Cane 3/b,GIARDINO,PIANO,0 -20%,MEDIO IMPASTO,3.480,6.502,2009-10-20,...,N/A,0.365648,0,0,0,0,-1,0.000000,"POLYGON ((11.34255 44.49084, 11.34256 44.49085...",0.000000


In [24]:
green_area.to_file(r'D:\Climate_Shelter_Index\isocrona\CSI')